In [86]:
import pandas as pd
import folium
from folium import IFrame

In [87]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Søre Goppollvegen,  2634 Fåvang", exactly_one=True)
print(location)
location = geolocator.geocode("2634 Fåvang", exactly_one=True)
print(location)

None
Fåvang, Ringebu, Oppland, 2634, Norge


In [88]:
df = pd.read_excel('LeveranseTilSlakteriLandbruket_Alle_Storfe.xlsx')

In [89]:
df.head()

,orgnr,navn,komnr,kalv_kg,ku_kg,kvige_kg,okse_kg,ungokse_kastrat_kg,ungku_kg,år,komnavn,organisasjonsform,forretningsadr,forradrpostnr,forradrpoststed,slaktsum_kg
0,969752042,STENUMGÅRD LUDVIG,520,0.0,2193.2,0.0,0.0,7368.1,1849.0,2013,Ringebu,ENK,STENUMGÅRD,2634,FÅVANG,11410.3
1,977548241,ELIN HOBBERSTAD,1119,0.0,3146.5,0.0,0.0,0.0,2041.1,2013,Hå,ENK,Nordsjøvegen 2460,4360,VARHAUG,5187.6
2,994914855,SØLVERØD SAMDRIFT DA,811,0.0,846.7,0.0,3832.0,15557.1,0.0,2013,Siljan,DA,Rustadveien 270,3748,SILJAN,20235.8
3,969163330,TOR ANDERS HAGA,1124,0.0,3292.8,0.0,0.0,7618.8,664.4,2013,Sola,ENK,Rudleholsvegen 62,4051,SOLA,11576.0
4,969085305,VIGRE OLAV JOHANNES,1119,0.0,2534.9,0.0,0.0,0.0,423.4,2013,Hå,ENK,Nordsjøvegen 2962,4362,VIGRESTAD,2958.3


In [90]:
df['address'] = df['forretningsadr'] + ", "+ df['forradrpostnr'].map(int).map(str)+ " " + df['forradrpoststed']

In [91]:
df['address'].head()

0              STENUMGÅRD, 2634 FÅVANG
1      Nordsjøvegen 2460, 4360 VARHAUG
2         Rustadveien 270, 3748 SILJAN
3         Rudleholsvegen 62, 4051 SOLA
4    Nordsjøvegen 2962, 4362 VIGRESTAD
Name: address, dtype: object

In [92]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

In [93]:
def get_lat_lon(r):
    print(r,end='\r')
    try:
        return geolocator.geocode(r)[-1]
    except:
        return (float('nan'),float('nan'))
        

In [128]:
df.shape

(46225, 17)

In [136]:
df2 = df.head(1000)

In [137]:
df2.loc[:,('lat')], df2.loc[:,('lon')] = zip(*df2.loc[:,('address')].map(get_lat_lon))

/home/patechoc/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/patechoc/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [138]:
df2.columns

Index(['orgnr', 'navn', 'komnr', 'kalv_kg', 'ku_kg', 'kvige_kg', 'okse_kg',
       'ungokse_kastrat_kg', 'ungku_kg', 'år', 'komnavn', 'organisasjonsform',
       'forretningsadr', 'forradrpostnr', 'forradrpoststed', 'slaktsum_kg',
       'address', 'lat', 'lon'],
      dtype='object')

In [139]:
df2[['lat','lon']].head()

,lat,lon
0,NaN,NaN
1,58.603929,5.635387
2,NaN,NaN
3,58.863475,5.668398
4,58.547745,5.690699


In [140]:
df3  = df2[df2['lat'].notnull()]
df3.head()
df3.shape


(295, 19)

In [150]:
df3.to_csv('farmers_dataFrame.csv')
df3.to_excel('farmers_dataFrame.xlsx')

In [151]:
import random

df3 = pd.read_csv('farmers_dataFrame.csv')

# Make a data frame with dots to show on the map
data = pd.DataFrame({
'lat':[-58, 2, 145, 30.32, -4.03, -73.57, 36.82, -38.5],
'lon':[-34, 49, -38, 59.93, 5.33, 45.52, -1.29, -12.97],
'name':['Buenos Aires', 'Paris', 'melbourne', 'St Petersbourg', 'Abidjan', 'Montreal', 'Nairobi', 'Salvador']
})

data = df3[['lat', 'lon','navn']]

# Make an empty map
# my_map = folium.Map(location=[60.4720, 8.4689], tiles="Mapbox Bright", zoom_start=2)
my_map = folium.Map(location=[60.4720, 8.4689], tiles="Stamen Terrain", zoom_start=5)


folium.TileLayer('openstreetmap').add_to(my_map)
# other mapping code (e.g. lines, markers etc.)
folium.LayerControl().add_to(my_map)

for i in range(0,len(data)):
    html="<h3> " + data.iloc[i]['navn'].title() + """</h3><br>
     <button class="button" onClick="window.open('http://www.google.com');"><span class="icon">Gå til rapporten</span></button>
    """
    iframe = IFrame(html=html, width=400, height=150)
    popup = folium.Popup(iframe, max_width=400)
    r = random.random()
    if r > 0.7:
        color = 'green'
    elif r > 0.3:
        color = "orange"
    else:
        color = 'red'
    folium.Marker([data.iloc[i]['lat'], data.iloc[i]['lon']], popup=popup, icon=folium.Icon(color=color)).add_to(my_map)
#     folium.Marker([data.iloc[i]['lat'], data.iloc[i]['lon']], popup=data.iloc[i]['navn']).add_to(my_map)

my_map.save('producerMap.html')
my_map

In [ ]:
from folium import IFrame
html="""
    <h1> This is a big popup</h1><br>
    With a few lines of code...
    <p>
    <code>
        from numpy import *<br>
        exp(-2*pi)
    </code>
    </p>
    """
iframe = IFrame(html=html, width=500, height=300)
popup = folium.Popup(iframe, max_width=2650)

folium.Marker([30,-100], popup=popup).add_to(m)
